In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
import torch
import wandb
import numpy as np
import pandas as pd
from transformers.trainer import Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score
from peft import get_peft_model, LoraConfig, TaskType
from transformers.training_args import TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification 

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pooyahatami83 (pooyahatami83-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}

train_df = pd.read_parquet("hf://datasets/hezarai/sentiment-dksf/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/hezarai/sentiment-dksf/" + splits["test"])

ds = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'test': Dataset.from_pandas(test_df)
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_name = 'xlm-roberta-base'
safe_model_name = "./models/" + model_name.replace('/','_').replace('-','_').lower() + '_lora'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=12,
        lora_alpha=24,
        lora_dropout=0.1,
        target_modules=[
          "query", "key", "value",
          "dense",
          "intermediate.dense",
          "output.dense"
        ],
        modules_to_save=["classifier"]
    )

peft_model = get_peft_model(model, lora_config)

print("\nTraining Parameter Analysis:")
print("=" * 60)

lora_params = 0
full_params = 0
frozen_params = 0

training_breakdown = {
    "lora_adapters": [],
    "full_training": [],
    "frozen": []
}

for name, param in model.named_parameters():
    if param.requires_grad:
        if "lora_" in name:
            lora_params += param.numel()
            training_breakdown["lora_adapters"].append(name)
        else:
            full_params += param.numel()
            training_breakdown["full_training"].append(name)
    else:
        frozen_params += param.numel()
        training_breakdown["frozen"].append(name)

total_params = lora_params + full_params + frozen_params

print(f"LoRA Adapter Parameters: {lora_params:,} ({lora_params/total_params*100:.2f}%)")
print(f"Full Training Parameters: {full_params:,} ({full_params/total_params*100:.2f}%)")
print(f"Frozen Parameters: {frozen_params:,} ({frozen_params/total_params*100:.2f}%)")
print(f"Total Trainable: {lora_params + full_params:,} ({(lora_params + full_params)/total_params*100:.2f}%)")


Training Parameter Analysis:
LoRA Adapter Parameters: 2,009,088 (0.72%)
Full Training Parameters: 592,899 (0.21%)
Frozen Parameters: 278,045,955 (99.07%)
Total Trainable: 2,601,987 (0.93%)


In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

peft_model = peft_model.to(device)
print(f"Model moved to {device} device")

Using device: cuda
Model moved to cuda device


In [9]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.rename_column("label", "labels")

Map:   0%|          | 0/28602 [00:00<?, ? examples/s]

Map:   0%|          | 0/2315 [00:00<?, ? examples/s]

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

training_args = TrainingArguments(
    output_dir=f"./{safe_model_name}/results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    warmup_steps=300,
    weight_decay=0.01,
    logging_dir=f"./{safe_model_name}/logs",
    logging_steps=50,
    eval_steps=300,
    save_steps=600,
    eval_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    fp16=True,
    report_to="wandb",
    run_name=safe_model_name,
    disable_tqdm=False,
)

In [13]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_ds["train"], # type: ignore
    eval_dataset=tokenized_ds["test"], # type: ignore
    compute_metrics=compute_metrics,
)

print("Starting training...")
trainer.train()


peft_model.save_pretrained(safe_model_name+"_model")
tokenizer.save_pretrained(safe_model_name+"_tokenizer")
print("Model and tokenizer saved!")

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss,Validation Loss,Accuracy,F1
300,0.749700,0.558654,0.787041,0.800383
600,0.615500,0.502855,0.806048,0.818684
900,0.564100,0.464940,0.816415,0.826540
1200,0.579400,0.439834,0.826350,0.832912
1500,0.555700,0.491904,0.803024,0.820845
1800,0.523300,0.451584,0.820734,0.831074
2100,0.485500,0.453323,0.827214,0.837105
2400,0.534800,0.437307,0.828942,0.837932


Model and tokenizer saved!


In [ ]:
y_true = []
y_pred = []

for batch in trainer.get_eval_dataloader():
    batch = {k: v.to(trainer.model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = trainer.model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    y_true.extend(batch["labels"].cpu().numpy())
    y_pred.extend(predictions.cpu().numpy())

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1107
           1       0.93      0.81      0.87      1032
           2       0.40      0.65      0.49       176

    accuracy                           0.83      2315
   macro avg       0.73      0.78      0.74      2315
weighted avg       0.85      0.83      0.84      2315

[[966  45  96]
 [116 839  77]
 [ 43  19 114]]


In [15]:
model_artifact = wandb.Artifact(
    name=safe_model_name,
    type="model",
    description="LoRA fine-tuned BERT variant for sentiment analysis"
)
model_artifact.add_dir(safe_model_name+"_model")
wandb.log_artifact(model_artifact)

wandb: Adding directory to artifact (./xlm_roberta_base_lora_model)... Done. 0.1s


<Artifact xlm_roberta_base_lora>